In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
from ark import io
from pathlib import Path
import spatialdata as sd
from spatialdata.models import C
from spatial_image import SpatialImage

/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


## 1. Convert Cohort Structure to Zarr Store

Load the example dataset's image data consisting of FOVs as a Spatial Data Object. Then write it as a Zarr Store to take advantage of efficient processing.

In [4]:
cohort_dir = Path("../data/example_dataset/image_data")


# list(cohort_dir.glob("*/"))
spatial_data: sd.SpatialData = io.load_cohort(cohort_dir)

In [5]:
spatial_data

SpatialData object with:
└── Images
      ├── 'fov0': SpatialImage[cyx] (22, 512, 512)
      ├── 'fov1': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov2': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov3': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov4': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov5': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov6': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov7': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov8': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov9': SpatialImage[cyx] (22, 1024, 1024)
      └── 'fov10': SpatialImage[cyx] (22, 1024, 1024)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images), fov1 (Images), fov2 (Images), fov3 (Images), fov4 (Images), fov5 (Images), fov6 (Images), fov7 (Images), fov8 (Images), fov9 (Images), fov10 (Images)

We can access any Image in the Spatial Data object by using the image name as a key in the `images` dictionary.

In [6]:
spatial_data.images.keys()

dict_keys(['fov4', 'fov3', 'fov10', 'fov2', 'fov5', 'fov0', 'fov7', 'fov9', 'fov8', 'fov6', 'fov1'])

In [7]:
spatial_data.images["fov0"]

<xarray.SpatialImage 'image' (c: 22, y: 512, x: 512)>
dask.array<_map_read_frame, shape=(22, 512, 512), dtype=float32, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Attributes:
    transform:  {'global': Identity }

In [8]:
# Print the channels of the first FOV in the cohort:
spatial_data.images["fov0"].c

<xarray.SpatialImage 'c' (c: 22)>
array(['CD3', 'CD4', 'CD8', 'CD14', 'CD20', 'CD31', 'CD45', 'CD68', 'CD163',
       'CK17', 'Collagen1', 'ECAD', 'Fibronectin', 'GLUT1', 'H3K9ac',
       'H3K27me3', 'HLADR', 'IDO', 'Ki67', 'PD1', 'SMA', 'Vim'], dtype='<U11')
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
Attributes:
    long_name:  c
    units:

Select a few channels from `fov0`

In [9]:
spatial_data.images["fov0"].sel({C: ["CD3", "CD4", "CD8"]})

<xarray.SpatialImage 'image' (c: 3, y: 512, x: 512)>
dask.array<getitem, shape=(3, 512, 512), dtype=float32, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Attributes:
    transform:  {'global': Identity }

Write the Spatial Data object to a Zarr Store

In [10]:
spatial_data.write(file_path="../data/cohorts/example_cohort.ome.zarr")

ValueError: The Zarr store already exists. Use `overwrite=True` to overwrite the store.

Load the Zarr Store as a Spatial Data Object

In [11]:
sdata = sd.read_zarr(store="../data/cohorts/example_cohort.ome.zarr/")

In [12]:
sdata.sel(elements=["fov0"])

SpatialData object with:
└── Images
      └── 'fov0': SpatialImage[cyx] (22, 512, 512)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images)

In [13]:
sdata.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[0, 0],
    max_coordinate=[1024, 1024],
    target_coordinate_system="global",
)

SpatialData object with:
└── Images
      ├── 'fov0': SpatialImage[cyx] (22, 512, 512)
      ├── 'fov1': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov2': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov3': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov4': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov5': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov6': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov7': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov8': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov9': SpatialImage[cyx] (22, 1024, 1024)
      └── 'fov10': SpatialImage[cyx] (22, 1024, 1024)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images), fov1 (Images), fov2 (Images), fov3 (Images), fov4 (Images), fov5 (Images), fov6 (Images), fov7 (Images), fov8 (Images), fov9 (Images), fov10 (Images)

In [14]:
sdata.images["fov0"]

<xarray.SpatialImage 'fov0' (c: 22, y: 512, x: 512)>
dask.array<from-zarr, shape=(22, 512, 512), dtype=float32, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Attributes:
    transform:  {'global': Identity }

In [15]:
nucs = ["H3K9ac", "H3K27me3"]
mems = ["CD14", "CD45", "ECAD"]

In [39]:
sdata.segmentation.run_deepcell(nucs=["H3K9ac", "H3K27me3"], mems=["CD14", "CD45", "ECAD"])

TypeError: 'list' object is not callable

In [ ]:
import xbatcher
from xbatcher import BatchAccessor

In [ ]:
for i in xbatcher.BatchGenerator(sdata.images["fov0"], input_dims={C: 8}):
    print(i)